In this notebook, we will be using the https://www.coingecko.com/en/api?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork1005-2022-01-01%3ECoinGecko API to create one of these candlestick graphs for Bitcoin. We will use the API to get the price data for 30 days with 24 observation per day, 1 per hour. We will find the max, min, open, and close price per day meaning we will have 30 candlesticks and use that to generate the candlestick graph. Although we are using the CoinGecko API we will use a Python client/wrapper for the API called PyCoinGecko. PyCoinGecko will make performing the requests easy and it will deal with the enpoint targeting.

In [1]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance
!pip install --upgrade nbformat

     ---------------------------------------- 0.0/62.8 kB ? eta -:--:--
     ---------------------------------------- 62.8/62.8 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/140.9 kB ? eta -:--:--
     ------------------------------ ------- 112.6/140.9 kB 2.2 MB/s eta 0:00:01
     -------------------------------------- 140.9/140.9 kB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/155.3 kB ? eta -:--:--
     -------------------------------------  153.6/155.3 kB 9.0 MB/s eta 0:00:01
     -------------------------------------- 155.3/155.3 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/97.1 kB ? eta -:--:--
     ---------------------------------------- 97.1/97.1 kB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/15.2 MB ? eta -:--:--
     ---------------------------------------- 0.2/15.2 MB 3.1 MB/s eta 0:00:05
     - -------------------------------------- 0.4/15.2 MB 3.5 MB/s eta 0:00:05


Import Library

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

Matplotlib is building the font cache; this may take a moment.


Lets start off by getting the data we need. Using the get_coin_market_chart_by_id(id, vs_currency, days). id is the name of the coin you want, vs_currency is the currency you want the price in, and days is how many days back from today you want.

In [3]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=30)

In [4]:
type(bitcoin_data)

dict

The response we get is in the form of a JSON which includes the price, market caps, and total volumes along with timestamps for each observation. We are focused on the prices so we will select that data.

In [7]:
bitcoin_price_data = bitcoin_data['prices']
bitcoin_price_data[0:5]

[[1677078112068, 24005.60066033856],
 [1677081625913, 23738.790027126597],
 [1677085365463, 23688.237675512464],
 [1677088919708, 23736.359875852024],
 [1677092430908, 23808.36149127166]]

Lets turn this data into a Pandas DataFrame.

In [8]:
data = pd.DataFrame(bitcoin_price_data, columns=['Timestamp', 'Price'])

Now that we have the DataFrame we will convert the timestamp to datetime and save it as a column called Date. We will map our unix_to_datetime to each timestamp and convert it to a readable datetime.

In [9]:
data['date'] = data['Timestamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))

Using this modified dataset we can now group by the Date and find the min, max, open, and close for the candlesticks.

In [10]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})

Finally we are now ready to use plotly to create our Candlestick Chart.

In [11]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'],
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'],
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()